In [7]:
movies_df = pd.read_csv('resources/data/movies.csv',sep = ',')
ratings_df = pd.read_csv('resources/data/ratings.csv')
ratings_df.drop(['timestamp'], axis=1,inplace=True)
ratings_df.head()

model=pickle.load(open('resources/models/algo.pkl', 'rb'))

print(model.predict(1,31))


user: 1          item: 31         r_ui = None   est = 3.05   {'was_impossible': False}


In [29]:

# Script dependencies
import pandas as pd
import numpy as np
import pickle
import copy
from surprise import Reader, Dataset
from surprise import SVD, NormalPredictor, BaselineOnly, KNNBasic, NMF
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

# Importing data
movies_df = pd.read_csv('resources/data/movies.csv',sep = ',')
ratings_df = pd.read_csv('resources/data/ratings.csv')
ratings_df.drop(['timestamp'], axis=1,inplace=True)

# We make use of an SVD model trained on a subset of the MovieLens 10k dataset.
model=pickle.load(open('resources/models/algo.pkl', 'rb'))

def prediction_item(item_id):
    """Map a given favourite movie to users within the
       MovieLens dataset with the same preference.

    Parameters
    ----------
    item_id : int
        A MovieLens Movie ID.

    Returns
    -------
    list
        User IDs of users with similar high ratings for the given movie.

    """
    # Data preprosessing
    reader = Reader(rating_scale=(0, 5))
    load_df = Dataset.load_from_df(ratings_df,reader)
    a_train = load_df.build_full_trainset()

    predictions = []
    for ui in a_train.all_users():
        predictions.append(model.predict(iid=item_id,uid=ui, verbose = False))
    return predictions

def pred_movies(movie_list):
    """Maps the given favourite movies selected within the app to corresponding
    users within the MovieLens dataset.

    Parameters
    ----------
    movie_list : list
        Three favourite movies selected by the app user.

    Returns
    -------
    list
        User-ID's of users with similar high ratings for each movie.

    """
    # Store the id of users
    id_store=[]
    # For each movie selected by a user of the app,
    # predict a corresponding user within the dataset with the highest rating
    for i in movie_list:
        predictions = prediction_item(item_id = i)
        predictions.sort(key=lambda x: x.est, reverse=True)
        # Take the top 10 user id's from each movie with highest rankings
        for pred in predictions[:100]:
            id_store.append(pred.uid)
    # Return a list of user id's
    return id_store

# !! DO NOT CHANGE THIS FUNCTION SIGNATURE !!
# You are, however, encouraged to change its content.  
def collab_model(movie_list,top_n=10):
    """Performs Collaborative filtering based upon a list of movies supplied
       by the app user.

    Parameters
    ----------
    movie_list : list (str)
        Favorite movies chosen by the app user.
    top_n : type
        Number of top recommendations to return to the user.

    Returns
    -------
    list (str)
        Titles of the top-n movie recommendations to the user.

    """

    indices = pd.Series(movies_df['title'])
    movie_ids = pred_movies(movie_list)

    df_init_users = ratings_df[ratings_df['userId']==movie_ids[0]]
    # df_init_users = ratings_df
    for i in movie_ids :
        # df_init_users=df_init_users.append(ratings_df[ratings_df['userId']==i])
        df_init_users = pd.concat([df_init_users,ratings_df[ratings_df['userId']==i]])
    # Getting the cosine similarity matrix
    cosine_sim = cosine_similarity(np.array(df_init_users), np.array(df_init_users))
    idx_1 = indices[indices == movie_list[0]].index[0]
    idx_2 = indices[indices == movie_list[1]].index[0]
    idx_3 = indices[indices == movie_list[2]].index[0]
    print(idx_1,idx_2,idx_3,"lol",len(cosine_sim))
    # Creating a Series with the similarity scores in descending order
    rank_1 = cosine_sim[idx_1]
    rank_2 = cosine_sim[idx_2]
    rank_3 = cosine_sim[idx_3]
    # Calculating the scores
    score_series_1 = pd.Series(rank_1).sort_values(ascending = False)
    score_series_2 = pd.Series(rank_2).sort_values(ascending = False)
    score_series_3 = pd.Series(rank_3).sort_values(ascending = False)
    
    # Appending the names of movies
    # listings = score_series_1.append(score_series_1).append(score_series_3).sort_values(ascending = False)
    listings = pd.concat([score_series_1,score_series_2,score_series_3])
    recommended_movies = []
    # Choose top 50
    top_50_indexes = list(listings.iloc[1:50].index)
    # Removing chosen movies
    top_indexes = np.setdiff1d(top_50_indexes,[idx_1,idx_2,idx_3])
    for i in top_indexes[:top_n]:
        recommended_movies.append(list(movies_df['title'])[i])
    return recommended_movies


In [30]:
Lists = ["Grumpier Old Men (1995)","Friday (1995)","Outbreak (1995)"]
List = ["Jumanji (1995)","Skylark (1941)","Sabotage (2014)"] 
top = collab_model(movie_list=Lists)
print(top)
for i in enumerate(top):
    print(i)

2 68 288 lol 45106
['Unzipped (1995)', 'Go Fish (1994)', 'Alien Escape (1995)', 'Twilight (1998)', 'Fitzcarraldo (1982)', 'Born Yesterday (1950)', 'Me, Myself & Irene (2000)', 'Couch in New York, A (1996)', 'Pledge, The (2001)', "How I Killed My Father (a.k.a. My Father and I) (Comment j'ai tué mon Père) (2001)"]
(0, 'Unzipped (1995)')
(1, 'Go Fish (1994)')
(2, 'Alien Escape (1995)')
(3, 'Twilight (1998)')
(4, 'Fitzcarraldo (1982)')
(5, 'Born Yesterday (1950)')
(6, 'Me, Myself & Irene (2000)')
(7, 'Couch in New York, A (1996)')
(8, 'Pledge, The (2001)')
(9, "How I Killed My Father (a.k.a. My Father and I) (Comment j'ai tué mon Père) (2001)")
